# Stellar Spectra

Stellar spectra can be generated by combining a `Stars` object with an ``EmissionModel``, translating the properties of the stellar populations (typically `initial_masses`, `ages` and `metallicities`) to a spectral energy distribution. 

These models are described in detail in the [emission model docs](../../emission_models/emission_models.rst). 
Here, we'll use an instance of a ``PacmanEmission`` model for demonstration purposes.

The following sections demonstrate the generation of *integrated* spectra (which is the same for both parametric and particle ``Stars``), and *per--particle* spectra. 


In [ ]:
from unyt import K, Msun, Myr

from synthesizer.emission_models import (
    AttenuatedEmission,
    Greybody,
    PacmanEmission,
)
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.grid import Grid
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.parametric import SFH, Stars, ZDist

tau_v = 0.5
# dust curve slope
alpha = -1.0
dust_curve = PowerLaw(slope=alpha)
dust_emission_model = Greybody(30 * K, 1.2)

grid_name = "test_grid"
grid = Grid(grid_name)

stellar_mass = 10**11 * Msun
sfh = SFH.Constant(max_age=100 * Myr)
metal_dist = ZDist.Normal(mean=0.01, sigma=0.05)

# Get the 2D star formation and metal enrichment history for the
# given SPS grid. This is (age, Z).
stars = Stars(
    grid.log10age,
    grid.metallicity,
    sf_hist=sfh,
    metal_dist=metal_dist,
    initial_mass=stellar_mass,
)

# Get the model
pacman = PacmanEmission(
    grid=grid,
    tau_v=tau_v,
    dust_curve=dust_curve,
    dust_emission=dust_emission_model,
)
print(pacman)

## Integrated spectra

To generate integrated spectra we simply call the components ``get_spectra`` method. This method will populate the component's ``spectra`` attribute with a dictionary containing [Sed objects](../emission_objects/sed_example.ipynb) for each spectra in the ``EmissionModel`` and will also return the spectra at the root of the ``EmissionModel``.

In [ ]:
# Get the spectra using a unified agn model (instantiated elsewhere)
spectra = stars.get_spectra(pacman)

We can plot the resulting spectra using the ``plot_spectra`` method.

In [ ]:
fig, ax = stars.plot_spectra(show=True, figsize=(6, 4))

The spectra returned by ``get_spectra`` is the "total" spectra at the root of the emission model.

In [ ]:
print(spectra)

However, all the spectra are stored within a dictionary under the ``spectra`` attribute on the relevant component.

In [ ]:
print(stars.spectra)

## Particle spectra

In this example we load some test particle data from CAMELS:

In [ ]:
from synthesizer import TEST_DATA_DIR

# Create stars component object
stars = load_CAMELS_IllustrisTNG(
    TEST_DATA_DIR,
    snap_name="camels_snap.hdf5",
    group_name="camels_subhalo.hdf5",
    physical=True,
)[1].stars

To generate a spectra for each star particle we use the same model, but we need to tell the model to produce a spectrum for each particle. This is done by setting the ``per_particle`` flag to ``True`` on the model.

In [ ]:
pacman.set_per_particle(True)

With that done we just call the same ``get_spectra`` method on the component, and the particle spectra will be stored in the ``particle_spectra`` attribute of the component.

In [ ]:
spectra = stars.get_spectra(pacman, verbose=True)

Again, the returned spectra is the "total" spectra from the root of the model.

In [ ]:
print(spectra)

While the spectra produced by ``get_particle_spectra`` are stored in a dictionary under the ``particle_spectra`` attribute.

In [ ]:
print(stars.particle_spectra)

### Integrating spectra

The integrated spectra are automatically produced alongside per particle spectra. However, if we wanted to explictly get the integrated spectra from the particle spectra we just generated (for instance if we had made some modification after generation), we can call the ``integrate_particle_spectra`` method.
This method will sum the individual spectra, and populate the ``spectra`` dictionary (overwriting whats already there!).

Note that we can also integrate individual spectra using the [``Sed.sum()`` method](../../sed/sed.ipynb).

In [ ]:
print(stars.spectra)
stars.integrate_particle_spectra()
print(stars.spectra)

fig, ax = stars.plot_spectra(show=True, figsize=(6, 4))

## Varying model parameters and reusing existing spectra

If we want vary one or more model parameters (e.g., dust attenuation parameters), we don't want to regenerate the whole model from scratch. Instead, we can create a new model that reuses an existing spectra on the emitter. To signal this we pass a string to the model instead of another `EmissionModel`. 

For our example of dust attenuation, this means passing a string to the ``apply_to`` argument of the ``AttenuatedEmission`` model. This string should match one of the keys in the emitter's ``spectra`` dictionary, here we use ``"reprocessed"``. When we run this it will extract the relevant spectra from the emitter and apply the attenuation to it without regenerating the original spectra.

In [ ]:
# Since we now want integrated spectra lets remove the per particle flag
pacman.set_per_particle(False)

spectra = {}
for tau_v in [0.1, 0.5, 1.0]:
    # Create a new model to modify the existing "intrinsic"
    attenuated = AttenuatedEmission(
        label=f"attenuated_tauv_{tau_v}",
        tau_v=tau_v,
        dust_curve=dust_curve,
        emitter="stellar",
        apply_to="reprocessed",
    )

    stars.get_spectra(attenuated)
    spectra[r"$\tau_v " f"= {tau_v}"] = stars.spectra[attenuated.label]

This is also applicable to the ``combine`` argument on ``Combinations`` and the various dependencies for generators (like intrinsic and attenuated spectra for energy balance dust emission models).

To see the variation above we can pass the dictionary we populated with the varied spectra to the `plot_spectra` function (where the dictionary keys will be used as labels). 

In [ ]:
from synthesizer.emissions import plot_spectra

plot_spectra(spectra, xlimits=(10**2.5, 10**5.5))

## Printing Used Parameters

During spectra generation, emission models cache the parameters they extract and use from the emitter. These cached parameters can be printed in a nicely formatted table to inspect which values were actually used by each model.

In [ ]:
# Print the cached parameters used by the models
stars.print_used_parameters()